In [18]:
import pandas as pd
link = "movie_dataset.csv"

df = pd.read_csv(link, sep= ',')
df

,Title,Director,Stars,IMDb-Rating,Category,Duration,Censor-board-rating,ReleaseYear
0,Top Gun: Maverick,JosephKosinski,"TomCruise, JenniferConnelly, MilesTeller, ValK...",8.6,"Action,Drama",130min,UA,2022
1,Everything Everywhere All at Once,"DanKwan,",", MichelleYeoh, StephanieHsu, KeHuyQuan, James...",8.3,"Action,Adventure,Comedy",139min,R,2022
2,The Batman,MattReeves,"RobertPattinson, ZoëKravitz, JeffreyWright, Co...",7.9,"Action,Crime,Drama",176min,UA,2022
3,Jurassic Park,StevenSpielberg,"SamNeill, LauraDern, JeffGoldblum, RichardAtte...",8.2,"Action,Adventure,Sci-Fi",127min,UA,1993
4,The Godfather,FrancisFordCoppola,"MarlonBrando, AlPacino, JamesCaan, DianeKeaton",9.2,"Crime,Drama",175min,A,1972
...,...,...,...,...,...,...,...,...
995,Vizontele,"YilmazErdogan,",", YilmazErdogan, DemetAkbag, AltanErkekli, Cem...",8.0,"Comedy,Drama",110min,NaN,2001
996,Sarfarosh,JohnMathewMatthan,"AamirKhan, NaseeruddinShah, SonaliBendre, Muke...",8.1,"Action,Drama,Thriller",174min,A,1999
997,Udaan,VikramadityaMotwane,"RajatBarmecha, RonitRoy, ManjotSingh, RamKapoor",8.1,Drama,134min,UA,2010
998,English Vinglish,GauriShinde,"Sridevi, AdilHussain, MehdiNebbou, PriyaAnand",7.8,"Comedy,Drama,Family",134min,U,2012


In [19]:
features = ['Category', 'Title', 'Stars', 'Director']

def combine_features(row):
    return row['Title']+' '+row['Category']+' '+row['Director']+' '+row['Stars']

for feature in features:
    df[feature] = df[feature].fillna('')

df['combined_features'] = df.apply(combine_features, axis = 1)

df['index'] = df.index

In [20]:
def get_title_from_index(index):
    return df[df.index == index]["Title"].values[0]
def get_index_from_title(Title):
    return df[df.Title == Title]["index"].values[0]

In [21]:
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
count_matrix = cv.fit_transform(df['combined_features'])
movie_user_likes = "The Avengers"
movie_index = get_index_from_title(movie_user_likes)

# Utilisation des indices filtrés pour l'entraînement de KNN
knn_model = NearestNeighbors(metric='cosine', algorithm='brute')
knn_model.fit(count_matrix)
_, indices = knn_model.kneighbors(count_matrix[movie_index], n_neighbors=11)

print("Top 10 similar movies to " + movie_user_likes + " are:\n")
for index in indices.flatten()[1:]:
    print(get_title_from_index(index))

Top 10 similar movies to The Avengers are:

Avengers: Infinity War
Captain America: Civil War
Captain America: The Winter Soldier
Avengers: Endgame
Serenity
Iron Man
The Fifth Element
The Terminator
Aliens
Star Wars: Episode VII - The Force Awakens


In [22]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#df['Category'] = df['Category'].fillna('')

# Création de la matrice des caractéristiques
tfidf = TfidfVectorizer()
count_matrix = tfidf.fit_transform(df['Title'] + ' ' + df['Director'] + ' ' + df['Stars'] + ' ' + df['Category'])

# Calcul de la similarité cosine
cosine_sim = cosine_similarity(count_matrix)

# Exemple d'une recommandation pour le film "The Avengers"
movie_user_likes = "The Avengers"
movie_index = df[df['Title'] == movie_user_likes].index[0]
similar_movies_cosine = cosine_sim[movie_index]

# Obtention des films similaires basés sur la similarité cosine
similar_movies_indices = similar_movies_cosine.argsort()[::-1][1:11]
similar_movies = df.loc[similar_movies_indices, 'Title']

# Affichage des films similaires
print(f"Top 10 similar movies to {movie_user_likes} are:\n")
for movie in similar_movies:
    print(movie)

Top 10 similar movies to The Avengers are:

Avengers: Infinity War
Avengers: Endgame
Captain America: Civil War
Captain America: The Winter Soldier
Serenity
Iron Man
Arrival
The Prestige
Her
The Fifth Element


In [23]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Création de la matrice des caractéristiques
tfidf = TfidfVectorizer()
count_matrix = tfidf.fit_transform(df['Title'] + ' ' + df['Director'] + ' ' + df['Stars'] + ' ' + df['Category'])

# Calcul de la similarité cosine
cosine_sim = cosine_similarity(count_matrix)

# Exemple d'une recommandation pour le film "The Avengers"
movie_user_likes = "The Avengers"
movie_index = df[df['Title'] == movie_user_likes].index[0]
similar_movies_cosine = cosine_sim[movie_index]

# Obtention des films similaires basés sur la similarité cosine
similar_movies_indices = similar_movies_cosine.argsort()[::-1][1:11]
similar_movies = df.loc[similar_movies_indices, 'Title']

# Entraînement du modèle de forêt aléatoire
random_forest_model = RandomForestClassifier()
random_forest_model.fit(count_matrix, df['Category'])

# Boucle sur les indices des films similaires
for index, movie in zip(similar_movies_indices, similar_movies):
    # Récupération des données du film
    movie_features = count_matrix[index].toarray()

    # Prédiction de la catégorie du film
    prediction = random_forest_model.predict(movie_features)[0]

    # Calcul de l'indice de confiance (probabilité) de la prédiction
    confidence = np.max(random_forest_model.predict_proba(movie_features))

    # Affichage du film recommandé et l'indice de confiance
    print("Movie Recommendation:", movie)
    print("Confidence:", confidence)
    print("---------------------------------------------\n")


c:\Users\morga\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\morga\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


Movie Recommendation: Avengers: Infinity War
Confidence: 0.89
---------------------------------------------

Movie Recommendation: Avengers: Endgame
Confidence: 0.81
---------------------------------------------

Movie Recommendation: Captain America: Civil War
Confidence: 0.93
---------------------------------------------

Movie Recommendation: Captain America: The Winter Soldier
Confidence: 0.86
---------------------------------------------

Movie Recommendation: Serenity
Confidence: 0.92
---------------------------------------------

Movie Recommendation: Iron Man
Confidence: 0.92
---------------------------------------------

Movie Recommendation: Arrival
Confidence: 0.76
---------------------------------------------

Movie Recommendation: The Prestige
Confidence: 0.8
---------------------------------------------

Movie Recommendation: Her
Confidence: 0.69
---------------------------------------------

Movie Recommendation: The Fifth Element
Confidence: 0.88
-----------------------